## Import Packages

In [1]:
import csv
import numpy as np
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

C:\Users\ar1\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Extract Data

In [2]:
def findCommunities(filename):
    communities = []
    with open(filename, 'r') as csvfile:
        filereader = csv.reader(csvfile)
        j = 0
        for i in filereader:
            if j > 0:
                if i[1] not in communities:
                    communities.append(i[1])
            j+=1
    return communities
def extractData(filename,communities,addstate = False,addcommunity = False):
    features = []
    xPositive=[]
    yPositive=[]
    xNegative=[]
    yNegative =[]
    with open(filename, 'r') as csvfile:
        filereader = csv.reader(csvfile)
        j = 0
        for i in filereader:
            if j > 0:
                data = i[5:len(i)-1]
                while '?' in data:
                    data[data.index('?')] = 0
                if addstate:
                    state = [0.0]*56
                    state[int(i[0])-1] = 1.0
                    data = data + state
                    features = features + ["state"+str(i) for i in range(56)]
                if addcommunity:
                    tmp = [0.0]*len(communities)
                    tmp[communities.index(i[1])] = 1.0
                    data = data + tmp
                    features = features + communities
                vect = np.array(data)
                #vect = np.array(i[3:len(i)-1])
                #x.append(vect.astype(np.float))
                if float(i[-1]) > 0.1:
                    xPositive.append(vect.astype(np.float))
                    yPositive.append(1)
                else:
                    xNegative.append(vect.astype(np.float))
                    yNegative.append(0)
            else:
                features = i[5:len(i)-1]
            j+=1
        """yPositive = np.array(yPositive)
        xPositive = np.array(xPositive)
        yNegative = np.array(yNegative)
        xNegative = np.array(xNegative)"""
        features = np.array(features)
        indxP = int(len(xPositive)*.6)
        indxN = int(len(xNegative)*.6)
        xTrain =np.array(xPositive[:indxP] + xNegative[:indxN])
        yTrain = np.array(yPositive[:indxP] + yNegative[:indxN])
        xTest = np.array(xPositive[indxP:] + xNegative[indxN:])
        yTest = np.array(yPositive[indxP:] + yNegative[indxN:])
        
        
            
            
        return xTrain,yTrain,xTest,yTest,features

## Calculate Label Percentage

In [3]:
def calcualtePercentage(labels):
    n = len(labels)
    sumTrue = sum(labels)*1.0
    sumFalse = (n - sumTrue)*1.0
    return sumTrue/n,sumFalse/n

## Create Model

In [4]:
def randomForest():
    return RandomForestClassifier.RandomForestClassifier(n_estimators=10, criterion='gini', max_depth=None, min_samples_split=2, 
                                       min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', 
                                       max_leaf_nodes=None, min_impurity_split=1e-07, bootstrap=True, oob_score=False, 
                                       n_jobs=1, random_state=None, verbose=0, warm_start=False, class_weight=None)

## Evaluation

In [5]:
def confusionMatrix(true,predicted,labels = [1,0]):
    return confusion_matrix(true,predicted,labels)

def find_confusion_matrix(actual,predicted,clabels):
    cm= []
    for i in clabels:
        tmp =[0]*len(clabels)
        
        for j in range(len(actual)):
            if actual[j]== i and actual[j] == predicted[j]:
                tmp[clabels.index(i)] += 1
            elif actual[j]== i and actual[j] != predicted[j]:
                tmp[clabels.index(predicted[j])] += 1
        cm.append(tmp)
    return np.array(cm)
def find_accuracy(matrix):
    return np.trace(matrix)*1.0/np.sum(matrix)
def find_precision(matrix):
    pres = []
    x = np.sum(matrix,axis=0)
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if i == j:
                pres.append(matrix[i][j]*1.0/x[i])
    return pres
def find_recall(matrix):
    rec = []
    x = np.sum(matrix,axis=1)
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if i == j:
                rec.append(matrix[i][j]*1.0/x[i])
    return rec
def evaluation(acutal,predicted,clabels=[1,0]):
    confmatrix = find_confusion_matrix(acutal,predicted,clabels)
    print ("Confusion Matrix")
    print (confmatrix)
    accuracy = find_accuracy(confmatrix)
    print ("Accuracy", accuracy)
    precision = find_precision(confmatrix)
    print ("Precision", precision)
    recall = find_recall(confmatrix)
    print ("Recall", recall)

## Cross Validation

In [6]:
def do_cross_validation(X, y,clf, n_folds=5):
    cv = KFold(len(y), n_folds, shuffle=True)
    accuracies = []
    i = 1
    for train_ind, test_ind in cv: 
        clf.fit(X[train_ind], y[train_ind])
        predictions = clf.predict(X[test_ind])
        print ("Fold %d Evaluation" %i)
        i +=1
        evaluation(y[test_ind],predictions)
        print ("\n")
        accuracies.append(accuracy_score(y[test_ind], predictions))
    avg = np.mean(accuracies)
    print ("The average Accuracy is ", avg)

### Read Data

In [7]:
filename = 'C:\\Users\\ar1\\Documents\ML\\Project\\Crime Prediction Data(1)\\Crime Prediction Data\\communities-crime-full.csv'
distintCommunities = findCommunities(filename)
XTrain,YTrain,XTest,YTest,Features = extractData(filename, distintCommunities)

### Percentage of Labels

In [8]:
print ("Train Percentage",calcualtePercentage(YTrain))
print ("Test Percentage",calcualtePercentage(YTest))

Train Percentage (0.62761506276150625, 0.3723849372384937)
Test Percentage (0.62703379224030042, 0.37296620775969963)


### Default Decision Tree Model

In [9]:
model= RandomForestClassifier(max_depth=None, min_samples_split=2,n_estimators=10)
model.fit(XTrain,YTrain)
predictedY=model.predict(XTest)
evaluation(YTest,predictedY)

Confusion Matrix
[[413  88]
 [ 53 245]]
Accuracy 0.823529411765
Precision [0.88626609442060089, 0.7357357357357357]
Recall [0.82435129740518964, 0.82214765100671139]


### 10 Fold Cross Validation on Default DecisionTree Model

In [10]:
do_cross_validation(XTrain,YTrain,model,10)

Fold 1 Evaluation
Confusion Matrix
[[52 10]
 [16 42]]
Accuracy 0.783333333333
Precision [0.76470588235294112, 0.80769230769230771]
Recall [0.83870967741935487, 0.72413793103448276]


Fold 2 Evaluation
Confusion Matrix
[[58 14]
 [10 38]]
Accuracy 0.8
Precision [0.8529411764705882, 0.73076923076923073]
Recall [0.80555555555555558, 0.79166666666666663]


Fold 3 Evaluation
Confusion Matrix
[[61  7]
 [ 7 45]]
Accuracy 0.883333333333
Precision [0.8970588235294118, 0.86538461538461542]
Recall [0.8970588235294118, 0.86538461538461542]


Fold 4 Evaluation
Confusion Matrix
[[64 14]
 [12 30]]
Accuracy 0.783333333333
Precision [0.84210526315789469, 0.68181818181818177]
Recall [0.82051282051282048, 0.7142857142857143]


Fold 5 Evaluation
Confusion Matrix
[[63 13]
 [10 34]]
Accuracy 0.808333333333
Precision [0.86301369863013699, 0.72340425531914898]
Recall [0.82894736842105265, 0.77272727272727271]


Fold 6 Evaluation
Confusion Matrix
[[62 17]
 [ 7 33]]
Accuracy 0.798319327731
Precision [0.898550724

### Top 10 important features 

In [11]:
coef = model.feature_importances_
topindx = np.argsort(coef)[::-1][:10]
print (Features[topindx])


['PctIlleg' 'PctPersDenseHous' 'PctFam2Par' 'FemalePctDiv' 'PctHousNoPhone'
 'pctWInvInc' 'PctKids2Par' 'TotalPctDiv' 'HousVacant' 'racePctWhite']
